In [1]:
# ## Install Coqui TTS
# ! pip install -U pip
# ! pip install TTS==0.8.0

#!pip install git+https://github.com/coqui-ai/TTS  # from Github


# !ls


In [2]:

!nvidia-smi
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
# BaseDatasetConfig: defines name, formatter and path of the dataset.

from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

from TTS.utils.audio import AudioProcessor
from TTS.tts.configs.shared_configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig
#from TTS.configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig#GlowTTSConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.speakers import SpeakerManager

from TTS.config import load_config
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.models.glow_tts import GlowTTS
from trainer import Trainer, TrainerArgs

import torch

Sun Oct 16 12:07:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:65:00.0 Off |                  Off |
| 33%   55C    P0    94W / 300W |     47MiB / 49140MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

/home/ansary/anaconda3/envs/mobassir/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# # !apt install -y espeak
# !apt install -y espeak-ng
!pwd

/home/ansary/Shabab


In [4]:
!ls bn_tts

IndicTTS_Phase2_Bengali_fem_Speaker1_mono
IndicTTS_Phase2_Bengali_male_Speaker1_mono


In [5]:

# output_path = "tts_train_dir"
# if not os.path.exists(output_path):
#     os.makedirs(output_path)

male = True
pretrained = True

pretrained_path = ''
if(pretrained):
    pretrained_path = '/home/ansary/Shabab/bn_GlowTTS'
if(male):
    meta_file = '/home/ansary/Shabab/bn_tts/IndicTTS_Phase2_Bengali_male_Speaker1_mono/mono/metadata_male.txt'
    root_path = '/home/ansary/Shabab/bn_tts/IndicTTS_Phase2_Bengali_male_Speaker1_mono/mono'
else:
    meta_file = '/home/ansary/Shabab/bn_tts/IndicTTS_Phase2_Bengali_fem_Speaker1_mono/mono/metadata_female.txt'
    root_path = '/home/ansary/Shabab/bn_tts/IndicTTS_Phase2_Bengali_fem_Speaker1_mono/mono'

def formatter(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the LJSpeech meta data file to TTS format
    https://keithito.com/LJ-Speech-Dataset/"""
    txt_file = meta_file
    items = []
    speaker_name = "ljspeech"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wav", cols[0] + ".wav")
            try:
                text = cols[1]
            except:
                print("not found")

            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


dataset_config = BaseDatasetConfig(
     meta_file_train=meta_file, path=os.path.join(root_path, "")
)


In [6]:
dataset_config

BaseDatasetConfig(name='', path='/home/ansary/Shabab/bn_tts/IndicTTS_Phase2_Bengali_male_Speaker1_mono/mono/', meta_file_train='/home/ansary/Shabab/bn_tts/IndicTTS_Phase2_Bengali_male_Speaker1_mono/mono/metadata_male.txt', ignored_speakers=None, language='', meta_file_val='', meta_file_attn_mask='')

In [7]:
# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config,formatter=formatter, eval_split=True)

 | > Found 6187 files in /home/ansary/Shabab/bn_tts/IndicTTS_Phase2_Bengali_male_Speaker1_mono/mono


In [8]:
print(len(train_samples),len(eval_samples))

6126 61


In [9]:
train_samples[0]

{'text': 'সকাল সাড়ে নটায় লিলুয়া রেল ওয়ার্কশপে,পরিত্যক্ত পাটের স্তুপে,আগুন লাগে\n',
 'audio_file': '/home/ansary/Shabab/bn_tts/IndicTTS_Phase2_Bengali_male_Speaker1_mono/mono/wav/train_bengalimale_01658.wav',
 'speaker_name': 'ljspeech',
 'root_path': '/home/ansary/Shabab/bn_tts/IndicTTS_Phase2_Bengali_male_Speaker1_mono/mono/',
 'language': ''}

In [10]:
# we use the same path as this script as our training folder.
output_path = '/home/ansary/Shabab/'

audio_config = BaseAudioConfig(
     sample_rate = 22050,
    resample =True
)

if(male):
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "তট৫ভিঐঋখঊড়ইজমএেঘঙসীঢ়হঞ‘ঈকণ৬ঁৗশঢঠ\u200c১্২৮দৃঔগও—ছউংবৈঝাযফ\u200dচরষঅৌৎথড়৪ধ০ুূ৩আঃপয়’নলো",
    punctuations = "-!,|.? ",
    )
else:
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "ইগং়’ুঃন১ঝূও‘ঊোছপফৈ৮ষযৎঢঈকঠিজ০৬ীটডএঅঋধচে২৩ণউয়ঢ়খলভৗসহ্ড়দথবঔাঞশরৌম—ঐআৃঘঙ\u200cঁ৪৫ত",
    punctuations = ".?-!|, ",
    )


# GlowTTSConfig: all model related values for training, validating and testing.

config = GlowTTSConfig(
    #use_speaker_embedding = True,
    batch_size=400,
    eval_batch_size=64,
    num_loader_workers=16,
    num_eval_loader_workers=16,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=2000,
    text_cleaner="collapse_whitespace",
    use_phonemes=False,
#     phoneme_language="bn",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=50,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    audio = audio_config,
    characters = characters_config,
    save_step=1000,
    cudnn_benchmark=True,
    test_sentences = [
        "পিপলস ইন্স্যুরেন্স অব চায়না ছেষট্টি বছর আগে ব্যবসা চালু করে।",
        "সোনার বাংলা আমি তোমায় ভালবাসি।"
    ]
)

In [11]:

ap = AudioProcessor.init_from_config(config)
ap.resample = True

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:True
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [12]:
ap.resample

True

In [13]:

tokenizer, config = TTSTokenizer.init_from_config(config)

In [14]:

model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [15]:
model

GlowTTS(
  (encoder): Encoder(
    (emb): Embedding(87, 192)
    (prenet): ResidualConv1dLayerNormBlock(
      (conv_layers): ModuleList(
        (0): Conv1d(192, 192, kernel_size=(5,), stride=(1,), padding=(2,))
        (1): Conv1d(192, 192, kernel_size=(5,), stride=(1,), padding=(2,))
        (2): Conv1d(192, 192, kernel_size=(5,), stride=(1,), padding=(2,))
      )
      (norm_layers): ModuleList(
        (0): LayerNorm()
        (1): LayerNorm()
        (2): LayerNorm()
      )
      (proj): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
    )
    (encoder): RelativePositionTransformer(
      (dropout): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0): RelativePositionMultiHeadAttention(
          (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))


In [16]:
# retrain = True
# continue_path = '/home/apsisdev/mobassir/data/tts/run-September-30-2022_10+55AM-0000000/'
# if(retrain):
#     config = load_config(os.path.join(continue_path, "config.json"))
#     print(config)
#     checkpoint = torch.load(os.path.join(continue_path, "best_model.pth")) #938
#     model.load_state_dict(checkpoint["model"])


In [17]:
# config.batch_size = 192
# config.eval_batch_size = 128
# config.num_loader_workers = 16
# config.num_eval_loader_workers = 16
# config.epochs = 5000

In [18]:
# use continue_path argument below for retraining model from last saved weight https://github.com/coqui-ai/Trainer/blob/main/trainer/trainer.py#L210

trainer = Trainer(
    TrainerArgs(continue_path = pretrained_path), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

fatal: not a git repository (or any of the parent directories): .git
 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 24
 | > Num. of Torch Threads: 12
 | > Torch seed: 5431
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Restoring from checkpoint_38000.pth ...
 > Restoring Model...
 > Restoring Optimizer...
 > Restoring Scaler...
 > Model restored from step 38000

 > Model has 28601809 parameters


In [ ]:
%%time

trainer.fit()

 > Restoring best loss from best_model_35921.pth ...
 > Starting with loaded last best loss -0.687639

 > EPOCH: 0/8000
 --> /home/ansary/Shabab/bn_GlowTTS

 > TRAINING (2022-10-16 12:07:16) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 6126
 | > Preprocessing samples
 | > Max text length: 114
 | > Min text length: 16
 | > Avg text length: 64.72233104799217
 | 
 | > Max audio length: 602438.0
 | > Min audio length: 107671.0
 | > Avg audio length: 281024.02579170745
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 61
 | > Preprocessing samples
 | > Max text length: 92
 | > Min text length: 40
 | > Avg text length: 64.52459016393442
 | 
 | > Max audio length: 373032.0
 | > Min audio length: 149142.0
 | > Avg audio length: 274765.0
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 12.13066 (+0.00000)
     | > avg_loss: -0.54667 (+0.00000)
     | > avg_log_mle: -0.73158 (+0.00000)
     | > avg_loss_dur: 0.18491 (+0.00000)


 > EPOCH: 1/8000
 --> /home/ansary/Shabab/bn_GlowTTS

 > TRAINING (2022-10-16 12:12:43) 


...



> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 6126
 | > Preprocessing samples
 | > Max text length: 114
 | > Min text length: 16
 | > Avg text length: 64.72233104799217
 | 
 | > Max audio length: 602438.0
 | > Min audio length: 107671.0
 | > Avg audio length: 281024.02579170745
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 61
 | > Preprocessing samples
 | > Max text length: 92
 | > Min text length: 40
 | > Avg text length: 64.52459016393442
 | 
 | > Max audio length: 373032.0
 | > Min audio length: 149142.0
 | > Avg audio length: 274765.0
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 12.19177 (+0.23531)
     | > avg_loss: -0.56309 (+0.00417)
     | > avg_log_mle: -0.73875 (+0.00740)
     | > avg_loss_dur: 0.17566 (-0.00323)


 > EPOCH: 9/8000
 --> /home/ansary/Shabab/bn_GlowTTS

 > TRAINING (2022-10-16 12:55:07) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 6126
 | > Preprocessing samples
 | > Max text length: 114
 | > Min text length: 16
 | > Avg text length: 64.72233104799217
 | 
 | > Max audio length: 602438.0
 | > Min audio length: 107671.0
 | > Avg audio length: 281024.02579170745
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
